# Qiskit Experiment Classes

This notebook demonstrates the new Experiment class interface outlines in the design doc [].

It includes implementions of full experiments for:
* `QSTExperiment` an example implementation of a basic experiment
* `ParallelExperiment` class for combining experiments on different qubits into a single experiment
* `BatchExperiment` class for combining arbitrary experiments into a single batch experiment

The parallel and batch experiments will be used to combine various QST experiments on different numbers of qubits

In [1]:
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator

# Import Experiment Classes
from qiskit.ignis.experiments.new import (
    BatchExperiment,
    ParallelExperiment,
    QSTExperiment
)

backend = QasmSimulator()

## Running single experiments

First run several individual QST experiments

In [2]:
# 1-qubit state preparation circuit
qc1 = QuantumCircuit(1)
qc1.h(0)

# Run QST for applying circuit to qubit-4
exp1 = QSTExperiment(qc1, [4])
data1 = exp1.run(backend)

# View result data
data1

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 08f5f73f-0462-4f81-a995-fa52ed697f93
Status: Complete
Qubits: [4]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.51269084+0.j        , 0.49982366-0.00390487j],
               [0.49982366+0.00390487j, 0.48730916+0.j        ]],
              dims=(2,))

In [3]:
# 1-qubit state preparation circuit
qc2 = QuantumCircuit(1)
qc2.x(0)

# Run QST for applying circuit to qubit-1
exp2 = QSTExperiment(qc2, [1])
data2 = exp2.run(backend)

# View result data
data2

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: bf1d3867-ef43-4546-91bc-fc9481f0262b
Status: Complete
Qubits: [1]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 7.62764875e-05+0.j        , -7.81130818e-03-0.00390565j],
               [-7.81130818e-03+0.00390565j,  9.99923724e-01+0.j        ]],
              dims=(2,))

In [4]:
# 2-qubit state preparation circuit
qc3 = QuantumCircuit(2)
qc3.h(0)
qc3.cx(0, 1)

# Run QST for applying circuit to qubits [3, 0]
exp3 = QSTExperiment(qc3, [3, 0])
data3 = exp3.run(backend)

# View result data
data3

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 144a38a6-e1e8-442a-8914-66d862d0d452
Status: Complete
Qubits: [3, 0]
Run Circuits: 9
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 0.5070038 +0.j        ,  0.00327309-0.00417704j,
                -0.00333746-0.00240245j,  0.49085603-0.00651908j],
               [ 0.00327309+0.00417704j,  0.00354779+0.j        ,
                -0.00278114-0.00208891j,  0.00150155+0.00953651j],
               [-0.00333746+0.00240245j, -0.00278114+0.00208891j,
                 0.00344396+0.j        , -0.00507296-0.00304422j],
               [ 0.49085603+0.00651908j,  0.00150155-0.00953651j,
                -0.00507296+0.00304422j,  0.48492582+0.j        ]],
              dims=(2, 2))

## Batch Experiments

Next we demonstrate batch experiments where several individual experiments are grouped into a single meta-experiment. Currently this just appends the circuits from each sub experiment sequentially, but in the future it should allow shuffling or interleaving circuits from individual experiments in the batch circuit list to be executed.

For a simple first demonstration we consider running multiple experiments on the same qubit (in this case the same experiment twice)

In [5]:
# Batch of two experiments both on qubit-4
batch_exp1 = BatchExperiment(2 * [exp1])
batch_data1 = batch_exp1.run(backend)

# View result
batch_data1

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: 08a03b41-4ce0-467a-bb40-726a44804f9e
Status: Complete
Qubits: [4]
Run Circuits: 6
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment']
- experiment_ids: ['83b13e28-7253-4b31-967b-28ead0c8dce0', '5efd3f09-0bd6-4415-ab3d-ea7a3645a638']
- experiment_qubits: [[4], [4]]

#### Viewing sub experiment data

The experiment data returned from a batched experiment also contains individual experiment data for each sub experiment which can be accessed using `experiment_data(index)`

In [6]:
# Print sub-experiment data
for i in range(2):
    print(batch_data1.experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 83b13e28-7253-4b31-967b-28ead0c8dce0
Status: Complete
Qubits: [4]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.52826124+0.j        , 0.49895709-0.01559241j],
               [0.49895709+0.01559241j, 0.47173876+0.j        ]],
              dims=(2,)) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 5efd3f09-0bd6-4415-ab3d-ea7a3645a638
Status: Complete
Qubits: [4]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.49511855+0.j       , 0.49986082+0.0107392j],
               [0.49986082-0.0107392j, 0.50488145+0.j       ]],
              dims=(2,)) 



### Batches on differing qubits

Batch experiments don't need to be on the same qubits, each individual one can be run on arbitrary qubits.

In [7]:
batch_exp2 = BatchExperiment([exp1, exp2, exp1, exp3])
batch_data2 = batch_exp2.run(backend)

# Batch data
batch_data2

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: bb75f733-4871-4906-a02b-1d102c1d3edc
Status: Complete
Qubits: [4, 1, 3, 0]
Run Circuits: 18
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment', 'QSTExperiment', 'QSTExperiment']
- experiment_ids: ['d5bfb61f-2b38-4699-b4a6-b0e63e41639e', 'f7e80e20-3bdd-4be5-88fc-874e5d7689eb', '4945695d-b290-482d-93d5-86f8b006c1b6', '37490ee5-de39-4ae7-b74a-3897770c78bd']
- experiment_qubits: [[4], [1], [4], [3, 0]]

#### View sub experiment data

In [8]:
for i in range(4):
    print(batch_data2.experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: d5bfb61f-2b38-4699-b4a6-b0e63e41639e
Status: Complete
Qubits: [4]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.49414121+0.j        , 0.49995042+0.00390586j],
               [0.49995042-0.00390586j, 0.50585879+0.j        ]],
              dims=(2,)) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: f7e80e20-3bdd-4be5-88fc-874e5d7689eb
Status: Complete
Qubits: [1]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 1.19166674e-04+0.j        , -1.95265951e-03+0.01073963j],
               [-1.95265951e-03-0.01073963j,  9.99880833e-01+0.j        ]],
              dims=(2,)) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 4945695d-b290-482d-93d5-8

## Parallel Experiments

Another kind of meta-experiment is a *parallel experiment*. This involves combing sub experiments on different qubits into a single experiment where each sub circuit is applied to different qubits in parallel. The total number of circuits in the parallel experiment will be equal to the largest number of circuits of the individual sub experiments.

Processing is done by marginalizing the joint measurements into the exected count dictionaries defined by the classical bit registers in the individaul experiments.

### Parallel tomography example
For our example we run parallel 1 qubit QST on qubits [4] and [1], and 2-qubit QST on qubits [3, 0]. This results in 9 circuits being executed -- the number required for the 2-qubit QST experiment

In [9]:
# Define a parallel experiment
par_exp = ParallelExperiment([exp1, exp2, exp3])
par_data = par_exp.run(backend)

# View data
par_data

---------------------------------------------------
Experiment: ParallelExperiment
Experiment ID: b81fafbe-e403-4a03-be0f-616ff897440f
Status: Complete
Qubits: [4, 1, 3, 0]
Run Circuits: 9
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment', 'QSTExperiment']
- experiment_ids: ['dd4c0959-9299-46e7-82a8-808e231821e7', 'f1032aa8-d70d-4c34-8a00-d03024d6c7ae', '13eaac69-1a31-4951-8a34-d0393249113a']
- experiment_qubits: [[4], [1], [3, 0]]

#### View sub experiment data

In [10]:
for i in range(4):
    print(batch_data2.experiment_data(i), '\n')

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: d5bfb61f-2b38-4699-b4a6-b0e63e41639e
Status: Complete
Qubits: [4]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[0.49414121+0.j        , 0.49995042+0.00390586j],
               [0.49995042-0.00390586j, 0.50585879+0.j        ]],
              dims=(2,)) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: f7e80e20-3bdd-4be5-88fc-874e5d7689eb
Status: Complete
Qubits: [1]
Run Circuits: 3
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- value: DensityMatrix([[ 1.19166674e-04+0.j        , -1.95265951e-03+0.01073963j],
               [-1.95265951e-03-0.01073963j,  9.99880833e-01+0.j        ]],
              dims=(2,)) 

---------------------------------------------------
Experiment: QSTExperiment
Experiment ID: 4945695d-b290-482d-93d5-8

## Complicated Meta Experiments

We can arbitrarily combine batch experiments and parallel experiments into nested metaexperiments (so long as the parallel experiment components are defined on different qubits of course).

For example we can consider the following contribed example involving multiple layers of batch and parallel tomography experiments:

In [11]:
big_exp = BatchExperiment([
            ParallelExperiment([exp3, ParallelExperiment([exp1, exp2])]),
            BatchExperiment([exp1, exp2]),
            exp3])
big_data = big_exp.run(backend)
big_data

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: cfc07d5d-b260-4cd8-b653-beaa7abaa30a
Status: Complete
Qubits: [3, 0, 4, 1]
Run Circuits: 24
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['ParallelExperiment', 'BatchExperiment', 'QSTExperiment']
- experiment_ids: ['d2a56987-27ba-481b-91cc-ca2b8e693059', 'a41a2824-33e9-4608-8508-d520c57a17ad', '06e10fe2-c18e-44c0-a8a1-867b4c7d66f6']
- experiment_qubits: [[3, 0, 4, 1], [4, 1], [3, 0]]

In [12]:
# print sub experiments
# We could also print the sub^2-experiments of the parallel and batch sub-experiments

for i in range(3):
    print(big_data.experiment_data(i), '\n')

---------------------------------------------------
Experiment: ParallelExperiment
Experiment ID: d2a56987-27ba-481b-91cc-ca2b8e693059
Status: Complete
Qubits: [3, 0, 4, 1]
Run Circuits: 9
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'ParallelExperiment']
- experiment_ids: ['ece0c742-5af0-4771-83f9-871c5b74b6e5', 'ba44c342-13db-42ef-9ef2-0d47e0d3b52d']
- experiment_qubits: [[3, 0], [4, 1]] 

---------------------------------------------------
Experiment: BatchExperiment
Experiment ID: a41a2824-33e9-4608-8508-d520c57a17ad
Status: Complete
Qubits: [4, 1]
Run Circuits: 6
Run Analysis: 1
---------------------------------------------------
Last Analysis Result
- experiment_types: ['QSTExperiment', 'QSTExperiment']
- experiment_ids: ['0ff2e6b8-2ea7-4c5e-8c12-fb545f08d451', 'aeef9e9a-c7a7-4b7c-8ec4-06fc413c068e']
- experiment_qubits: [[4], [1]] 

---------------------------------------------------
Experiment: QS